## 데이터 로드

In [ ]:
from pathlib import Path
import numpy as np
import pandas as pd

# ====== 데이터 로드 ======
train_candidates = [
    Path("../data/29757_train_merged.csv"),
    Path("../data/1000_train_merged.csv"),
    Path("../data/10000_train_merged.csv"),
]
test_candidates = [
    Path("../data/29757_test_merged.csv"),
    Path("../data/1000_test_merged.csv"),
    Path("../data/10000_test_merged.csv"),
]

def resolve_path(candidates: list[Path]) -> Path:
    for p in candidates:
        if p.exists():
            return p
    raise FileNotFoundError(f"No dataset found in: {candidates}")

train_df = pd.read_csv(resolve_path(train_candidates))
test_df = pd.read_csv(resolve_path(test_candidates))

In [ ]:
import matplotlib.pyplot as plt

target_col = "died_in_icu"

datasets = {
    "train_proc": train_df,
    "test_proc": test_df,
}

fig, axes = plt.subplots(1, 2, figsize=(10, 6), sharey=True)

for ax, (name, df) in zip(axes, datasets.items()):
    counts = df[target_col].value_counts().sort_index()
    ratios = counts / counts.sum()

    ax.bar(ratios.index.astype(str), ratios.values, color=["tab:blue", "tab:red"])
    ax.set_xlabel(target_col)
    ax.set_title(f"Class Ratio in {name}")

    for i, v in enumerate(ratios.values):
        ax.text(i, v + 0.01, f"{v:.2%}", ha="center")

axes[0].set_ylabel("Ratio")
plt.tight_layout()
plt.show()


## 전처리 함수 설정

In [ ]:
# ====== 기본 설정 ======
target_col = "died_in_icu"
possible_group_cols = ["patientunitstayid", "patient_id"]
possible_time_cols = ["observationoffset"]

group_col = next((c for c in possible_group_cols if c in train_df.columns), None)
time_col = next((c for c in possible_time_cols if c in train_df.columns), None)
if group_col is None or time_col is None:
    raise ValueError("patient id or time column not found")

numeric_cols = train_df.select_dtypes(include=["number"]).columns
exclude = {target_col, "patient_id", "patientunitstayid", "observationoffset"}
base_cols = [c for c in numeric_cols if c not in exclude]

# ====== 함수 ======
def add_missing_and_time_features(df: pd.DataFrame, base_cols: list[str]) -> pd.DataFrame:
    df = df.sort_values([group_col, time_col]).copy()
    for col in base_cols:
        miss = df[col].isna().astype(int)
        last_time = df[time_col].where(df[col].notna()).groupby(df[group_col]).ffill()
        tsince = df[time_col] - last_time
        tsince = tsince.fillna(df[time_col])
        #df[f"{col}_miss"] = miss
        #df[f"{col}_tsince"] = tsince
    return df

def impute_base(df: pd.DataFrame, base_cols: list[str], medians: pd.Series) -> pd.DataFrame:
    df = df.sort_values([group_col, time_col]).copy()
    df[base_cols] = df.groupby(group_col, sort=False)[base_cols].ffill()
    df[base_cols] = df[base_cols].fillna(medians)
    return df

def sample_k_per_stay(df, group_col, time_col, k=10):
    df = df.sort_values([group_col, time_col]).copy()
    def pick_rows(g):
        n = len(g)
        if n <= k:
            return g
        idx = np.linspace(0, n - 1, k, dtype=int)
        return g.iloc[idx]
    return df.groupby(group_col, group_keys=False).apply(pick_rows)

def balance_ratio(df, target_col="died_in_icu", ratio_pos=0.08, random_state=42):
    pos = df[df[target_col] == 1]
    neg = df[df[target_col] == 0]
    desired_neg = int(len(pos) * (1 - ratio_pos) / ratio_pos)

    if desired_neg >= len(neg):
        desired_pos = int(len(neg) * ratio_pos / (1 - ratio_pos))
        pos = pos.sample(n=desired_pos, random_state=random_state)
    else:
        neg = neg.sample(n=desired_neg, random_state=random_state)

    return pd.concat([pos, neg]).sample(frac=1, random_state=random_state).reset_index(drop=True)

def sample_total_rows_by_group(df, group_col, n_total=1000, random_state=42):
    rng = np.random.default_rng(random_state)
    groups = {gid: g.index.to_numpy() for gid, g in df.groupby(group_col)}
    group_ids = list(groups.keys())
    rng.shuffle(group_ids)
    for gid in group_ids:
        rng.shuffle(groups[gid])

    picks = []
    while len(picks) < n_total and group_ids:
        next_group_ids = []
        for gid in group_ids:
            if len(picks) >= n_total:
                break
            idxs = groups[gid]
            if len(idxs) == 0:
                continue
            picks.append(idxs[0])
            groups[gid] = idxs[1:]
            if len(groups[gid]) > 0:
                next_group_ids.append(gid)
        group_ids = next_group_ids

    result = df.loc[picks].sample(frac=1, random_state=random_state).reset_index(drop=True)
    return result



## 최종 데이터셋 구성

In [ ]:
# ====== 사용 피처 ======

feature_cols = base_cols #파생변수 + ID + 타겟 + 시간변수 제외



# ====== 파이프라인: group 기준 랜덤 1000개 샘플  ======
train_proc = sample_total_rows_by_group(train_df, group_col, n_total=1000, random_state=42)
test_proc = sample_total_rows_by_group(test_df, group_col, n_total=1000, random_state=42)

# ====== 파이프라인: stayid별 10개 샘플  ======
train_proc = sample_k_per_stay(train_df, group_col, time_col, k=10)
test_proc = sample_k_per_stay(test_df, group_col, time_col, k=10)

# ====== 파생변수 생성 & 추가======
# train_proc = add_missing_and_time_features(train_proc, base_cols)
# test_proc = add_missing_and_time_features(test_proc, base_cols)

# ====== 결측치 처리 ======

train_medians = train_proc[base_cols].median()
train_proc = impute_base(train_proc, base_cols, train_medians)
test_proc = impute_base(test_proc, base_cols, train_medians)

# ====== 클래스 비율 맞추기 (train만) ======
train_proc = balance_ratio(train_proc, target_col=target_col, ratio_pos=0.08)
test_proc = balance_ratio(test_proc, target_col=target_col, ratio_pos=0.08)

# ====== 데이터셋 구성 ======
X_train = train_proc[feature_cols]
y_train = train_proc[target_col]

X_test = test_proc[feature_cols]
y_test = test_proc[target_col]

In [ ]:
X_train.shape

## 불균형 전처리 비교

In [ ]:
import matplotlib.pyplot as plt

target_col = "died_in_icu"

datasets = {
    "train_proc": train_proc,
    "test_proc": test_proc,
}

fig, axes = plt.subplots(1, 2, figsize=(10, 6), sharey=True)

for ax, (name, df) in zip(axes, datasets.items()):
    counts = df[target_col].value_counts().sort_index()
    ratios = counts / counts.sum()

    ax.bar(ratios.index.astype(str), ratios.values, color=["tab:blue", "tab:red"])
    ax.set_xlabel(target_col)
    ax.set_title(f"Class Ratio in {name}")

    for i, v in enumerate(ratios.values):
        ax.text(i, v + 0.01, f"{v:.2%}", ha="center")

axes[0].set_ylabel("Ratio")
plt.tight_layout()
plt.show()


## 모델 학습

In [ ]:
import numpy as np
import pandas as pd

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer

# 클래스 불균형 비율 계산

scale_pos_weight = 92/8

models = {
    "LR": Pipeline([
      #  ("imputer", SimpleImputer(strategy="median")),
        ("scaler", StandardScaler()),
        ("clf", LogisticRegression(max_iter=1000)),
    ]),
    "RF": RandomForestClassifier(
        n_estimators=400,
        random_state=42,
        n_jobs=-1,
        class_weight="balanced",
    ),
}

# XGBoost
try:
    import xgboost as xgb
    models["XGBoost"] = xgb.XGBClassifier(
        n_estimators=400,
        max_depth=5,
        learning_rate=0.05,
        subsample=0.9,
        colsample_bytree=0.9,
        eval_metric="logloss",
        random_state=42,
        n_jobs=-1,
        scale_pos_weight=scale_pos_weight,
    )
except Exception:
    pass

# LightGBM
try:
    import lightgbm as lgb
    models["LightGBM"] = lgb.LGBMClassifier(
        n_estimators=400,
        learning_rate=0.05,
        num_leaves=31,
        random_state=42,
        scale_pos_weight=scale_pos_weight,
    )
except Exception:
    pass


def aggregate_patient_scores(df: pd.DataFrame, scores: np.ndarray):
    tmp = df[[group_col, target_col]].copy()
    tmp["score"] = scores
    y_patient = tmp.groupby(group_col)[target_col].max()
    s_patient = tmp.groupby(group_col)["score"].mean()
    return y_patient, s_patient

pred_scores = {}

for name, model in models.items():
    model.fit(X_train, y_train)

    if hasattr(model, "predict_proba"):
        scores = model.predict_proba(X_test)[:, 1]
    else:
        raw = model.decision_function(X_test)
        scores = 1 / (1 + np.exp(-raw))

    pred_scores[name] = scores

print("models used:", list(pred_scores.keys()))
print("test n:", len(y_test))


## 성능평가

In [ ]:
# 시각화: ROC + Calibration + Decision Curve (row-level)
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.calibration import calibration_curve
import numpy as np

fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# 1) ROC
ax = axes[0]
for name, scores in pred_scores.items():
    fpr, tpr, _ = roc_curve(y_test, scores)
    auc = roc_auc_score(y_test, scores)
    ax.plot(fpr, tpr, label=f"{name} (AUC={auc:.3f})")
ax.plot([0, 1], [0, 1], "k--", alpha=0.5)
ax.set_xlabel("1 - Specificity")
ax.set_ylabel("Sensitivity")
ax.set_title("ROC Curves (row-level)")
ax.legend(loc="lower right")

# 2) Calibration
ax = axes[1]
for name, scores in pred_scores.items():
    prob_true, prob_pred = calibration_curve(y_test, scores, n_bins=10, strategy="uniform")
    ax.plot(prob_pred, prob_true, marker="o", label=name)
ax.plot([0, 1], [0, 1], "k--", alpha=0.5, label="Ideal")
ax.set_xlabel("Mean predicted probability")
ax.set_ylabel("Fraction of positives")
ax.set_title("Calibration Curves (row-level)")
ax.legend(loc="lower right")

# 3) Decision Curve
ax = axes[2]
thresholds = np.linspace(0.01, 0.99, 99)

def decision_curve(y_true, scores):
    n = len(y_true)
    net_benefit = []
    for t in thresholds:
        preds = scores >= t
        tp = ((preds == 1) & (y_true == 1)).sum()
        fp = ((preds == 1) & (y_true == 0)).sum()
        nb = (tp / n) - (fp / n) * (t / (1 - t))
        net_benefit.append(nb)
    return np.array(net_benefit)

for name, scores in pred_scores.items():
    nb = decision_curve(np.array(y_test), scores)
    ax.plot(thresholds, nb, label=name)

prevalence = np.mean(y_test)
nb_all = prevalence - (1 - prevalence) * (thresholds / (1 - thresholds))
ax.plot(thresholds, nb_all, "k--", alpha=0.4, label="Treat all")
ax.plot(thresholds, np.zeros_like(thresholds), "k:", alpha=0.6, label="Treat none")

ax.set_xlabel("Threshold probability")
ax.set_ylabel("Net benefit")
ax.set_title("Decision Curve Analysis (row-level)")
ax.legend(loc="upper right")

plt.tight_layout()
plt.show()


In [ ]:
# Confusion Matrix 서브플롯 (환자 단위 집계 기준)
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import math
import numpy as np

n_models = len(pred_scores)
n_cols = 3
n_rows = math.ceil(n_models / n_cols)

fig, axes = plt.subplots(n_rows, n_cols, figsize=(5 * n_cols, 4 * n_rows))
axes = np.array(axes).reshape(-1)

for ax, (name, scores) in zip(axes, pred_scores.items()):
    y_pred = (scores >= 0.5).astype(int)
    cm = confusion_matrix(y_test, y_pred)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm)
    disp.plot(ax=ax, cmap="Blues", colorbar=False)
    ax.set_title(f"{name}")

for ax in axes[n_models:]:
    ax.axis("off")

plt.tight_layout()
plt.show()


In [ ]:
from __future__ import annotations

import numpy as np
import pandas as pd

from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    roc_auc_score, average_precision_score
)

def evaluate_row_level(
    pred_scores: dict[str, np.ndarray | pd.Series],
    y_true: np.ndarray | pd.Series,
    threshold: float = 0.5,
) -> pd.DataFrame:
   
    y_true = np.asarray(y_true).astype(int)

    results = []
    for name, scores in pred_scores.items():
        y_score = np.asarray(scores).astype(float)
        y_pred = (y_score >= threshold).astype(int)

        acc = accuracy_score(y_true, y_pred)
        prec = precision_score(y_true, y_pred, zero_division=0)
        rec = recall_score(y_true, y_pred, zero_division=0)
        f1 = f1_score(y_true, y_pred, zero_division=0)

        if np.unique(y_true).size < 2:
            auc = np.nan
            ap = np.nan
        else:
            auc = roc_auc_score(y_true, y_score)
            ap = average_precision_score(y_true, y_score)

        results.append({
            "model": name,
            "N_rows": len(y_true),
            "pos_rate": float(y_true.mean()),
            "threshold": threshold,
            "ACC": acc,
            "PREC": prec,
            "REC": rec,
            "F1": f1,
            "AUC": auc,
            "PR-AUC": ap,
        })

    out = (
        pd.DataFrame(results)
        .set_index("model")
        .sort_values(["PR-AUC", "AUC"], ascending=False)
    )
    return out



metrics_df = evaluate_row_level(
    pred_scores=pred_scores,
    y_true=y_test,
    threshold=0.5
)

print(metrics_df.to_string(float_format=lambda x: f"{x:.4f}"))


## SHAP 분석

In [ ]:
# SHAP 분석 + 중요 피처 Top-N (row-level, 현재 models 기준)
import shap
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

X_shap = X_test.sample(n=min(1000, len(X_test)), random_state=42)
top_n = 30

for name, model in models.items():
    try:
        if name in ["RF", "XGBoost", "LightGBM"]:
            explainer = shap.TreeExplainer(model)
            shap_values = explainer.shap_values(X_shap)
        else:
            bg = X_shap.sample(100, random_state=42)
            explainer = shap.KernelExplainer(model.predict_proba, bg)
            shap_values = explainer.shap_values(X_shap, nsamples=100)

        if isinstance(shap_values, list) and len(shap_values) > 1:
            sv = shap_values[1]
        else:
            sv = shap_values

        mean_abs = np.abs(sv).mean(axis=0)
        imp = pd.Series(mean_abs, index=X_shap.columns).sort_values(ascending=False)

        print(f"\n{name} Top {top_n} features:")
        print(imp.head(top_n))

        plt.figure(figsize=(6, 4))
        imp.head(top_n).sort_values().plot(kind="barh")
        plt.title(f"{name} SHAP Feature Importance (Top {top_n})")
        plt.tight_layout()
        plt.show()

        shap.summary_plot(sv, X_shap, show=True, plot_type="bar")
        shap.summary_plot(sv, X_shap, show=True)

    except Exception as e:
        print(f"SHAP failed for {name}: {e}")
